In [125]:
import math
import re
from functools import reduce
from pathlib import Path
from itertools import permutations

In [54]:
test_input_1 = """[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
"""

def read_input(number_string):
    return [eval(line) for line in number_string.strip().split("\n")]

assert read_input("[1, 2]") == [[1, 2]]

assert read_input(
    """[1, 2]
[3, 4]""") == [[1, 2], [3, 4]]

assert read_input("""[1, 2, [3, 4]]
[5, 6]
[[8, 9], 0]""") == [[1, 2, [3, 4]],[5, 6], [[8, 9], 0]]


In [96]:
def add_to_the_left(number, left):
    if number is None:
        return left
    if isinstance(left, int):
        return left + number
    sub_left, sub_right = left
    return [sub_left, add_to_the_left(number, sub_right)]

assert add_to_the_left(1, 1) == 2
assert add_to_the_left(1, [2, 3]) == [2, 4]
assert add_to_the_left(1, [[2, 3], 4]) == [[2, 3], 5]
assert add_to_the_left(1, [2, [3, 4]]) == [2, [3, 5]]
assert add_to_the_left(1, [[2, 3], [4, 5]]) == [[2, 3], [4, 6]]
assert add_to_the_left(None, 1) == 1

In [97]:
def add_to_the_right(number, right):
    if number is None:
        return right
    if isinstance(right, int):
        return right + number
    sub_left, sub_right = right
    return [add_to_the_right(number, sub_left), sub_right]

assert add_to_the_right(1, 1) == 2
assert add_to_the_right(1, [2, 3]) == [3, 3]
assert add_to_the_right(1, [[2, 3], 4]) == [[3, 3], 4]
assert add_to_the_right(1, [2, [3, 4]]) == [3, [3, 4]]
assert add_to_the_right(1, [[2, 3], [4, 5]]) == [[3, 3], [4, 5]]
assert add_to_the_right(None, 1) == 1

In [110]:
def explode2(x, n=4):
    if isinstance(x, int):
        return False, None, x, None
    if n == 0:
        return True, x[0], 0, x[1]
    a, b = x
    exp, left, a, right = explode2(a, n - 1)
    if exp:
        return True, left, [a, add_to_the_right(right, b)], None
    exp, left, b, right = explode2(b, n - 1)
    if exp:
        return True, None, [add_to_the_left(left, a), b], right
    return False, None, x, None


def explode(element, level=1):
    if isinstance(element, int):
        return False, None, element, None
    left, right = element
    if level > 4:
        return True, left, 0, right
    exploded, left_left, left, left_right = explode(left, level+1)
    if exploded:
        return True, left_left, [left, add_to_the_right(left_right, right)], None    
    exploded, right_left, right, right_right = explode(right, level+1)
    if exploded:
        return True, None, [add_to_the_left(right_left, left), right], right_right
    return False, None, [left, right], None
    
assert explode(1) == (False, None, 1, None)
assert explode([1, 2]) == (False, None, [1, 2], None)
assert explode([[1, 2], 3]) == (False, None, [[1, 2], 3], None)
assert explode([[[1, 2], 3], 4]) == (False, None, [[[1, 2], 3], 4], None)
assert explode([[[[1, 2], 3], 4], 5]) == (False, None, [[[[1, 2], 3], 4], 5], None)
assert explode([[[[[9,8],1],2],3],4]) == (True, 9, [[[[0,9],2],3],4], None)
assert explode([7,[6,[5,[4,[3,2]]]]]) == (True, None, [7,[6,[5,[7,0]]]], 2)
assert explode([[6,[5,[4,[3,2]]]],1]) == (True, None, [[6,[5,[7,0]]],3], None)
assert explode([[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]) == (True, None, [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]], None)
assert explode([[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]) == (True, None, [[3,[2,[8,0]]],[9,[5,[7,0]]]], 2)

In [111]:
def split(number):
    if isinstance(number, int):
        if number > 9:
            return [math.floor(number/2), math.ceil(number/2)]
        return number
    
    left, right = number
    new_left = split(left)
    if new_left != left:
        return [new_left, right]
    return [left, split(right)]

assert split(9) == 9
assert split(10) == [5, 5]
assert split(11) == [5, 6]
assert split(12) == [6, 6]
assert split([[[[0,7],4],[15,[0,13]]],[1,1]]) == [[[[0,7],4],[[7,8],[0,13]]],[1,1]]
assert split([[[[0,7],4],[[7,8],[0,13]]],[1,1]]) == [[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]

In [113]:
def snail_reduce(number, verbose=False):
    if verbose:
        print(number)
    last_number = None
    while number != last_number:
        last_number = number[:]
        _, _, number, _ = explode(number)
        if number != last_number:
            if verbose:
                print(f"X {number}")
            continue
        number = split(number)
        if verbose:
                print(f"S {number}")
    return number

assert snail_reduce([[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]) == [[[[0,7],4],[[7,8],[6,0]]],[8,1]]

In [117]:
def snail_add(a, b, verbose=False):
    return snail_reduce([a, b], verbose=verbose)

assert snail_add([[[[4,3],4],4],[7,[[8,4],9]]], [1,1]) == [[[[0,7],4],[[7,8],[6,0]]],[8,1]]
assert snail_add(
    [[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]],
    [7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
) == [[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]

assert snail_add(
    [[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]],
    [[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
) == [[[[6,7],[6,7]],[[7,7],[0,7]]],[[[8,7],[7,7]],[[8,8],[8,0]]]]


assert snail_add(
    [[[[6,7],[6,7]],[[7,7],[0,7]]],[[[8,7],[7,7]],[[8,8],[8,0]]]],
    [[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
) == [[[[7,0],[7,7]],[[7,7],[7,8]]],[[[7,7],[8,8]],[[7,7],[8,7]]]]

assert snail_add(
    [[[[7,0],[7,7]],[[7,7],[7,8]]],[[[7,7],[8,8]],[[7,7],[8,7]]]],
    [7,[5,[[3,8],[1,4]]]]
) == [[[[7,7],[7,8]],[[9,5],[8,7]]],[[[6,8],[0,8]],[[9,9],[9,0]]]]

assert snail_add(
    [[[[7,7],[7,8]],[[9,5],[8,7]]],[[[6,8],[0,8]],[[9,9],[9,0]]]],
    [[2,[2,2]],[8,[8,1]]]
) == [[[[6,6],[6,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]]

assert snail_add(
    [[[[6,6],[6,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],
    [2,9]
) == [[[[6,6],[7,7]],[[0,7],[7,7]]],[[[5,5],[5,6]],9]]

assert snail_add(
    [[[[6,6],[7,7]],[[0,7],[7,7]]],[[[5,5],[5,6]],9]],
    [1,[[[9,3],9],[[9,0],[0,7]]]]
) == [[[[7,8],[6,7]],[[6,8],[0,8]]],[[[7,7],[5,0]],[[5,5],[5,6]]]]

assert snail_add(
    [[[[7,8],[6,7]],[[6,8],[0,8]]],[[[7,7],[5,0]],[[5,5],[5,6]]]],
    [[[5,[7,4]],7],1]
) == [[[[7,7],[7,7]],[[8,7],[8,7]]],[[[7,0],[7,7]],9]]

assert snail_add(
    [[[[7,7],[7,7]],[[8,7],[8,7]]],[[[7,0],[7,7]],9]],
    [[[[4,2],2],6],[8,7]]
) == [[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]

In [119]:
assert reduce(snail_add, ([1,1], [2,2], [3,3], [4,4])) == [[[[1,1],[2,2]],[3,3]],[4,4]]
assert reduce(snail_add, ([1,1], [2,2], [3,3], [4,4], [5,5])) == [[[[3,0],[5,3]],[4,4]],[5,5]]
assert reduce(snail_add, ([1,1], [2,2], [3,3], [4,4], [5,5], [6,6])) == [[[[5,0],[7,4]],[5,5]],[6,6]]

In [120]:
def magnitude(number):
    if isinstance(number, int):
        return number
    left, right = number
    return magnitude(left) * 3 + magnitude(right) * 2

assert magnitude([[1,2],[[3,4],5]]) == 143
assert magnitude([[[[0,7],4],[[7,8],[6,0]]],[8,1]]) == 1384
assert magnitude([[[[1,1],[2,2]],[3,3]],[4,4]]) == 445
assert magnitude([[[[3,0],[5,3]],[4,4]],[5,5]]) == 791
assert magnitude([[[[5,0],[7,4]],[5,5]],[6,6]]) == 1137
assert magnitude([[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]) == 3488

In [122]:
test_numbers = read_input(test_input_1)
snail_sum = reduce(snail_add, test_numbers)
assert snail_sum == [[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]
assert magnitude(snail_sum) == 4140 

In [124]:
snail_numbers = read_input(Path("input_1.txt").read_text())
snail_sum = reduce(snail_add, snail_numbers)
magnitude(snail_sum)

4088

In [132]:
max_magnitude = 0
for a, b in permutations(snail_numbers, 2):
    max_magnitude =  max(max_magnitude, magnitude(snail_add(a, b)))
print(max_magnitude)

4536
